In [3]:
import numpy as np
import torch
import torch.nn as nn
from konlpy.tag import Okt
import gensim
import os

# ---------------------------------------------------------
# 1. 형태소 분석 및 단어 사전(Vocabulary) 구축
# ---------------------------------------------------------
jvm_path = r'C:\Program Files\Java\jdk-25.0.2\bin\server\jvm.dll'
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-25.0.2'   # 꼭 추가

okt = Okt()
reviews = [
    "웨이팅이 너무 길었지만 존맛탱이네요! 가성비 최고.",
    "직원들이 불친절하고 맛도 그냥 그랬어요. 재방문 의사 없음"
]

# 토큰화 진행
tokenized_reviews = [okt.morphs(review, stem=True) for review in reviews]

# 데이터셋에 등장한 모든 단어를 모아 고유 인덱스 부여
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
for tokens in tokenized_reviews:
    for word in tokens:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

vocab_size = len(word_to_idx)
print(f"구축된 단어 사전 크기: {vocab_size}")

# ---------------------------------------------------------
# 2. 사전 학습된 Word2Vec 로드 및 Embedding Matrix 생성
# ---------------------------------------------------------
# 주의: 사전에 학습된 ko.bin 또는 ko.model 파일이 다운로드 되어 있어야 합니다.
# (예: Kyubyong의 한국어 Word2Vec 모델)
print("Word2Vec 모델 로딩 중...")
try:
    # gensim을 이용해 가벼운 모델 로드
    word2vec_model = gensim.models.Word2Vec.load('') 
    embed_dim = word2vec_model.vector_size
except FileNotFoundError:
    print("모델 파일이 없어 임시 차원(100)으로 진행합니다.")
    word2vec_model = {} # 실제 환경에서는 다운로드한 모델이 들어갑니다.
    embed_dim = 100

# 모델의 가중치를 담을 빈 행렬 생성 (vocab_size x embed_dim)
embedding_matrix = np.zeros((vocab_size, embed_dim))

# 다이닝코드 단어 사전을 순회하며 Word2Vec에서 벡터값을 가져옴
oov_count = 0
for word, i in word_to_idx.items():
    if word in word2vec_model:
        # 사전 학습된 벡터가 있으면 가져옴
        embedding_matrix[i] = word2vec_model[word]
    else:
        # 사전에 없는 단어(OOV)는 정규분포를 따르는 무작위 벡터로 초기화
        # <PAD>는 0벡터로 유지
        if word != "<PAD>":
            embedding_matrix[i] = np.random.normal(scale=0.6, size=(embed_dim,))
            oov_count += 1

print(f"임베딩 매트릭스 생성 완료. OOV(사전에 없는 단어) 개수: {oov_count}")
embedding_matrix_tensor = torch.FloatTensor(embedding_matrix)

# ---------------------------------------------------------
# 3. PyTorch DeepCoNN 모델에 주입
# ---------------------------------------------------------
class DeepCoNN(nn.Module):
    def __init__(self, embedding_matrix_tensor, num_filters=100, kernel_size=3):
        super(DeepCoNN, self).__init__()
        
        # from_pretrained를 사용하여 우리가 만든 행렬을 모델의 Embedding 층으로 만듦
        # 식당 도메인에 맞게 학습되도록 freeze=False로 설정 (가중치 업데이트 허용)
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix_tensor, 
            freeze=False, 
            padding_idx=0
        )
        
        embed_dim = embedding_matrix_tensor.shape[1]
        
        # User Network (CNN) 예시
        self.user_cnn = nn.Sequential(
            nn.Conv1d(embed_dim, num_filters, kernel_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )

    def forward(self, x):
        # x: (Batch, Seq_len)
        embedded = self.embedding(x) # (Batch, Seq_len, Embed_dim)
        embedded = embedded.permute(0, 2, 1) # CNN 입력을 위해 차원 변경: (Batch, Embed_dim, Seq_len)
        out = self.user_cnn(embedded)
        return out.squeeze(2)

# 모델 생성 테스트
model = DeepCoNN(embedding_matrix_tensor)
print("DeepCoNN 임베딩 레이어 연동 완료!")

구축된 단어 사전 크기: 25
Word2Vec 모델 로딩 중...
모델 파일이 없어 임시 차원(100)으로 진행합니다.
임베딩 매트릭스 생성 완료. OOV(사전에 없는 단어) 개수: 24
DeepCoNN 임베딩 레이어 연동 완료!


In [50]:
from gensim.models import KeyedVectors
# 압축을 푼 ko.bin 파일의 경로를 지정합니다.
print("Kyubyong Word2Vec 로딩 중...")
ft_model = KeyedVectors.load_word2vec_format(
    "C:/Users/hanan/Downloads/cc.ko.300.vec.gz", 
    binary=False, 
    limit=50000)
print("로딩 완료!")

# 단어 벡터 확인 예시
vector = ft_model['맛집']
print(vector.shape) # 보통 (200,) 차원을 가집니다.

Kyubyong Word2Vec 로딩 중...
로딩 완료!
(300,)


# demo model


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
config = Config()
set_seed(config.random_seed)

data = pd.read_csv('diningcode_data_crawling_20260125_1542.csv')

le = LabelEncoder()
data['user_id'] = le.fit_transform(data['user_name'])
data['item_id'] = le.fit_transform(data['item_name'])

data['user_rating'] = data['user_rating'].str.replace('점','').astype(float)
data = data.drop_duplicates()

In [26]:
# 리뷰 개수를 세기 위해 user_id를 기준으로 그룹화
user_review_counts = data.groupby('user_id').size()

# 리뷰 개수가 2개 이상인 user_id만 필터링
users_with_multiple_reviews = user_review_counts[user_review_counts >= 2].index

# 해당 user_id를 가진 데이터만 필터링
filtered_data = data[data['user_id'].isin(users_with_multiple_reviews)]

filtered_data

,Unnamed: 0,item_name,item_area,item_avg_rating,item_spec_area,user_name,user_tot_avg_rating,user_tot_rating_num,user_tot_follow_num,user_rating,user_query,taste,price,service,menu,date,user_id,item_id
1,1,서령 본점,남대문,4.6,서울특별시 중구 남대문로5가 120 1층,\r\n오또잇\r\n,3.9,278,2,5.0,먹어본 평양냉면 중에 제일 맛있었습니다 다른 곳들은 좀 짜다는 느낌이 강했는데 서령...,맛: 좋음,가격: 보통,응대: 좋음,"서령 순면, 항정 제육 반 접시(100g)",2025년 7월 31일,2826,72
2,2,서령 본점,남대문,4.6,서울특별시 중구 남대문로5가 120 1층,\r\n와퍼\r\n,4.1,494,27,5.0,서울에서 가장 맛있는 평양냉면집.\r\n깔끔하고 슴슴한데 이렇게 맛있을 수가 없다....,맛: 좋음,가격: 만족,응대: 좋음,서령 순면,2025년 8월 17일,2866,72
4,4,서령 본점,남대문,4.6,서울특별시 중구 남대문로5가 120 1층,\r\n어먹행먹\r\n,4.2,209,5,5.0,서령 순면(평양냉면) 슴슴한데 은은한 특유의 육수맛이 있어요. 여기는 서령초라는 걸...,맛: 좋음,가격: 만족,응대: 좋음,"서령 순면, 항정 제육 반 접시(100g)",2025년 5월 1일,2735,72
5,5,서령 본점,남대문,4.6,서울특별시 중구 남대문로5가 120 1층,\r\n다코미식가\r\n먹죽귀\r\n,3.9,1339,669,5.0,중구 시청역 남대문 근처에 위치한\r\n서령 다녀왔어요예전에 강화도에 있을 때에도 ...,맛: 좋음,가격: 만족,응대: 좋음,서령 순면,2025년 2월 1일,1470,72
6,6,서령 본점,남대문,4.6,서울특별시 중구 남대문로5가 120 1층,\r\n쿨제이\r\n,3.7,118,4,2.0,평냉계에서 많은 사람들한테 회자되는 '서령'을 갔다왔어요. 결론적으로 제 취향이 아...,맛: 보통,가격: 불만,응대: 보통,"서령 순면, 소주, 접시만두 반 접시(3개), 항정 제육 반 접시(100g)",2025년 8월 13일,3440,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12622,12622,연교,연남동,4.4,서울특별시 마포구 연남동 383-95 1층,\r\nmatziplist\r\n,3.4,1187,23,4.0,모임하기 좋은 개별적인 공간있어서 좋아보였다. 소룡퍼 맛은 좀 서운했는데 다른것들이...,맛: 좋음,가격: 만족,응대: 보통,샤오롱바오,2020년 3월 30일,691,91
12623,12623,연교,연남동,4.4,서울특별시 마포구 연남동 383-95 1층,\r\nsue\r\n,3.8,95,5,4.0,어향가지 대박. 소스가 새콤달콤하고 너무 맛있다. 동파육은 쏘쏘. .....,맛: 좋음,가격: 보통,응대: 보통,"동파육, 어향가지",2020년 6월 16일,843,91
12624,12624,연교,연남동,4.4,서울특별시 마포구 연남동 383-95 1층,\r\n먼지구동이\r\n,4.7,130,0,5.0,으이이 다 너무너무 맛있어요\r\n또가야지 망두도 면 종류도 다 맛잇어요 ...,맛: 좋음,가격: 만족,응대: 좋음,성쟁바오,2023년 2월 3일,2040,91
12625,12625,연교,연남동,4.4,서울특별시 마포구 연남동 383-95 1층,\r\nJung Yoon Ha\r\n,1.0,1,0,1.0,실수로 오픈되어있는 문 들어갔다 민망해하며 나왔습니다. 오픈시간 잘못 안 제 실수도...,맛: 부족,가격: 불만,응대: 나쁨,없음,2022년 7월 30일,187,91


In [12]:
"""
import pandas as pd
import numpy as np
from collections import Counter
# 한국어 형태소 분석기가 필요하다면 konlpy의 Okt나 Mecab을 추천합니다.
# from konlpy.tag import Okt
# okt = Okt()

# 1. 사용자/아이템별 리뷰 통합
user_reviews = data.groupby('user_id')['user_query'].apply(lambda x: ' '.join(x)).reset_index()
item_reviews = data.groupby('item_id')['user_query'].apply(lambda x: ' '.join(x)).reset_index()

# 2. 간단한 토큰화 및 단어 사전(Vocabulary) 생성 함수
def build_vocab(text_series, max_vocab_size=10000):
    words = []
    for text in text_series:
        # 실제 환경에서는 형태소 분석기(okt.morphs)를 사용하는 것이 성능에 훨씬 좋습니다.
        words.extend(text.split()) 
    
    counter = Counter(words)
    vocab = {word: i+2 for i, (word, _) in enumerate(counter.most_common(max_vocab_size))}
    vocab['<PAD>'] = 0 # 패딩용
    vocab['<UNK>'] = 1 # 모르는 단어용
    return vocab

# 3. 텍스트를 정수 인덱스로 변환하고 패딩하는 함수
def text_to_sequence(text_series, vocab, max_length=500):
    sequences = []
    for text in text_series:
        seq = [vocab.get(word, vocab['<UNK>']) for word in text.split()]
        # Truncating & Padding
        if len(seq) > max_length:
            seq = seq[:max_length]
        else:
            seq = seq + [vocab['<PAD>']] * (max_length - len(seq))
        sequences.append(seq)
    return np.array(sequences)

# 단어 사전 구축 및 시퀀스 변환 (사용자, 아이템 전체 리뷰를 합쳐서 하나의 사전을 만드는 것이 일반적입니다)
all_texts = pd.concat([user_reviews['user_query'], item_reviews['user_query']])
vocab = build_vocab(all_texts)

user_sequences = text_to_sequence(user_reviews['user_query'], vocab)
item_sequences = text_to_sequence(item_reviews['user_query'], vocab)
"""

"\nimport pandas as pd\nimport numpy as np\nfrom collections import Counter\n# 한국어 형태소 분석기가 필요하다면 konlpy의 Okt나 Mecab을 추천합니다.\n# from konlpy.tag import Okt\n# okt = Okt()\n\n# 1. 사용자/아이템별 리뷰 통합\nuser_reviews = data.groupby('user_id')['user_query'].apply(lambda x: ' '.join(x)).reset_index()\nitem_reviews = data.groupby('item_id')['user_query'].apply(lambda x: ' '.join(x)).reset_index()\n\n# 2. 간단한 토큰화 및 단어 사전(Vocabulary) 생성 함수\ndef build_vocab(text_series, max_vocab_size=10000):\n    words = []\n    for text in text_series:\n        # 실제 환경에서는 형태소 분석기(okt.morphs)를 사용하는 것이 성능에 훨씬 좋습니다.\n        words.extend(text.split()) \n    \n    counter = Counter(words)\n    vocab = {word: i+2 for i, (word, _) in enumerate(counter.most_common(max_vocab_size))}\n    vocab['<PAD>'] = 0 # 패딩용\n    vocab['<UNK>'] = 1 # 모르는 단어용\n    return vocab\n\n# 3. 텍스트를 정수 인덱스로 변환하고 패딩하는 함수\ndef text_to_sequence(text_series, vocab, max_length=500):\n    sequences = []\n    for text in text_series:\n        seq = [voca

In [27]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from konlpy.tag import Okt
from tqdm import tqdm

jvm_path = r'C:\Program Files\Java\jdk-25.0.2\bin\server\jvm.dll'
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-25.0.2'   # 꼭 추가


okt = Okt()

# --- 하이퍼파라미터 설정 ---
MAX_VOCAB_SIZE = 30000  # 단어장 최대 크기 (빈도수 상위 5만개)
MAX_DOC_LENGTH = 550   # 하나의 문서(리뷰 모음)에 들어갈 최대 단어 수

# 1. 유저별, 아이템(식당)별로 리뷰 텍스트 하나로 합치기
print("유저 및 식당별 리뷰 통합 중...")
user_docs = data.groupby('user_id')['user_query'].apply(lambda x: ' '.join(map(str, x))).reset_index()
item_docs = data.groupby('item_id')['user_query'].apply(lambda x: ' '.join(map(str, x))).reset_index()

print('형태소 분석 및 Tokenization processing')
tqdm.pandas(desc='user docs tokenization')
user_docs['tokenized'] = user_docs['user_query'].progress_apply(lambda x: okt.morphs(x,stem=True))
tqdm.pandas(desc='item docs tokenization')
item_docs['tokenized'] = item_docs['user_query'].progress_apply(lambda x: okt.morphs(x,stem=True))


# 2. 토큰화 및 단어장(Vocabulary) 생성
print("단어장 생성 중...")
all_tokens =[]
for tokens in user_docs['tokenized']:
    all_tokens.extend(tokens)
for tokens in item_docs['tokenized']:
    all_tokens.extend(tokens)

word_counts = Counter(all_tokens)

# 빈도수 순으로 단어 정렬 및 인덱스 부여 (0은 PAD, 1은 UNK)
vocab = {'<PAD>': 0, '<UNK>': 1}
for i, (word, count) in enumerate(word_counts.most_common(MAX_VOCAB_SIZE)):
    vocab[word] = i + 2  # 0과 1은 이미 차지했으므로 2부터 시작

print(f"완성된 단어장 크기: {len(vocab)}")

# 3. 텍스트를 정수 시퀀스로 변환하고 패딩(Padding)하는 함수
def text_to_padded_sequence(text, vocab, max_length):
    # 단어가 사전에 없으면 <UNK>(1)로 처리
    seq = [vocab.get(word, vocab['<UNK>']) for word in text.split()]
    
    # 길이에 맞게 자르거나(Truncate) 패딩(Padding)
    if len(seq) > max_length:
        return seq[:max_length] # 최대 길이보다 길면 자르기
    else:
        return seq + [vocab['<PAD>']] * (max_length - len(seq)) # 짧으면 0으로 채우기

# 4. 유저와 아이템의 문서를 정수 시퀀스로 변환하여 딕셔너리로 저장
print("정수 시퀀스 변환 및 패딩 적용 중...")
user_seq_dict = {}
for _, row in user_docs.iterrows():
    user_seq_dict[row['user_id']] = text_to_padded_sequence(row['user_query'], vocab, MAX_DOC_LENGTH)

item_seq_dict = {}
for _, row in item_docs.iterrows():
    item_seq_dict[row['item_id']] = text_to_padded_sequence(row['user_query'], vocab, MAX_DOC_LENGTH)

print("전처리 완료! user_seq_dict와 item_seq_dict가 생성되었습니다.")

유저 및 식당별 리뷰 통합 중...
형태소 분석 및 Tokenization processing


item docs tokenization: 100%|██████████| 160/160 [00:34<00:00,  4.62it/s]


단어장 생성 중...
완성된 단어장 크기: 11446
정수 시퀀스 변환 및 패딩 적용 중...
전처리 완료! user_seq_dict와 item_seq_dict가 생성되었습니다.


In [ ]:
import torch.nn as nn
# pip install gensim
from gensim.models import KeyedVectors, Word2Vec

# 1. 사전 학습된 한국어 임베딩 모델 로드 (Korean fastText-from Meta)
word2vec_model = KeyedVectors.load_word2vec_format("C:/Users/hanan/Downloads/cc.ko.300.vec.gz",binary=False,limit=30000)

EMBED_DIM = 300 
vocab_size = len(vocab) # 이전 단계에서 만든 단어 사전의 크기

# 2. PyTorch Embedding 레이어에 넣을 빈 가중치 행렬 생성
# 형태: (단어 사전 크기, 임베딩 차원)
embedding_matrix = np.zeros((vocab_size, EMBED_DIM))

# 3. 단어 사전(vocab)을 순회하며 가중치 행렬 채우기
for word, idx in vocab.items():
    if word in word2vec_model:
        # 사전 학습된 모델에 단어가 존재하면 해당 벡터를 가져옴
        embedding_matrix[idx] = word2vec_model[word]
    else:
        # 모델에 없는 단어(OOV)는 정규분포를 따르는 랜덤 벡터로 초기화하거나 0으로 둡니다.
        if word == '<PAD>':
            embedding_matrix[idx] = np.zeros(EMBED_DIM)
        else:
            embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBED_DIM,))

# Numpy 배열을 PyTorch Tensor로 변환
embedding_tensor = torch.FloatTensor(embedding_matrix)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

class DiningCodeDataset(Dataset):
    def __init__(self, df, user_seq_dict, item_seq_dict):
        self.users = df['user_id'].values
        self.items = df['item_id'].values
        self.ratings = df['user_rating'].values  # 예측해야 할 정답 평점
        
        self.user_seq_dict = user_seq_dict
        self.item_seq_dict = item_seq_dict

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        user_id = self.users[idx]
        item_id = self.items[idx]
        rating = self.ratings[idx]

        # 딕셔너리에서 미리 만들어둔 정수 시퀀스를 가져옴
        u_doc = self.user_seq_dict[user_id]
        i_doc = self.item_seq_dict[item_id]

        # PyTorch 모델 입력용 텐서로 변환 (타입 변환이 매우 중요합니다)
        u_doc_tensor = torch.tensor(u_doc, dtype=torch.long)
        i_doc_tensor = torch.tensor(i_doc, dtype=torch.long)
        rating_tensor = torch.tensor(rating, dtype=torch.float32)

        return u_doc_tensor, i_doc_tensor, rating_tensor

# 데이터로더 생성 (한 번에 64개씩 묶어서 모델에 전달)
BATCH_SIZE = 64
dataset = DiningCodeDataset(data, user_seq_dict, item_seq_dict)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DeepCoNN(nn.Module):
    def __init__(self,config,embedding_matrix):
        super(DeepCoNN,self).__init__()
        self.config = config

        #1.Embedding layer
        vocab_size,embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.embedding.weight = nn.Parameter(
            torch.FloatTensor(embedding_matrix)
        )
        self.embedding.weight.requires_grad = True

        #2-1.User Network
        self.user_cnn = nn.Conv1d(
            in_channels=embedding_dim,
            out_channels=config.num_filters,
            kernel_size=config.kernel_size,
            padding=1
        )
        self.user_fc = nn.Linear(config.num_filters,config.latent_dim)

        #2-2. Item Network
        self.item_cnn = nn.Conv1d(
            in_channels=embedding_dim,
            out_channels=config.num_filters,
            kernel_size=config.kernel_size,
            padding=1
        )
        self.item_fc = nn.Linear(config.num_filters,config.latent_dim)

        #Dropout
        self.dropout = nn.Dropout(config.dropout_rate)

        #3.FM Layer (2nd-order latent vectors)
        self.fm_linear = nn.Linear(config.latent_dim*2,1)
        
        self.fm_V = nn.Parameter(
            torch.rand(config.latent_dim*2,config.fm_k)
        )
        self.global_bias = nn.Parameter(torch.zeros(1))

    def forward(self,user_doc,item_doc):
        """
        Args:
            user_doc: (batch_size,max_doc_length)
            item_doc: (batch_size,max_doc_length)

        Returns:
            rating: (batch_size,)
        """

        #User_Net
        user_emb = self.embedding(user_doc) # (B,L,E)
        user_emb = user_emb.transpose(1,2) # (B,E,L) - conv1d input format

        user_conv = F.relu(self.user_cnn(user_emb)) 
        user_pool = F.max_pool1d(user_conv,kernel_size=user_conv.size(2))
        user_pool = user_pool.squeeze(2)

        user_latent = F.relu(self.user_fc(user_pool))
        user_latent = self.dropout(user_latent)

        #item_Net
        item_emb = self.embedding(item_doc) # (B,L,E)
        item_emb = item_emb.transpose(1,2) # (B,E,L) - conv1d input format

        item_conv = F.relu(self.item_cnn(item_emb)) 
        item_pool = F.max_pool1d(item_conv,kernel_size=item_conv.size(2))
        item_pool = item_pool.squeeze(2)

        item_latent = F.relu(self.item_fc(item_pool))
        item_latent = self.dropout(item_latent)

        #concatenate
        z = torch.cat([user_latent,item_latent],dim=1)

        #FM layer
        linear_term = self.fm_linear(z)

        interactions = torch.mm(z,self.fm_V)
        interactions_squared = torch.mm(z**2,self.fm_V**2)

        quadratic_term = 0.5*torch.sum(
            interactions**2-interactions_squared,dim=1,keepdim=True
        )

        #Predict
        rating = self.global_bias + linear_term.squeeze(1) + quadratic_term.squeeze(1)
        
        return rating

class Config:

    max_doc_length = 550
    embedding_dim = 300

    "Model Parameters"
    num_filters = 100
    kernel_size = 3
    latent_dim = 50
    fm_k = 8

    "Traning"

    batch_size = 64
    num_epochs = 50
    learning_rate = 0.002
    dropout_rate = 0.5

    "Other"
    device='cuda'
    random_seed=42

In [ ]:
import torch.nn as nn
import torch.optim as optim

config = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")
model = DeepCoNN(config=config, embedding_matrix=embedding_matrix).to(device)

# 평점 예측(회귀) 문제이므로 손실 함수는 MSE(Mean Squared Error) 사용
criterion = nn.MSELoss()

# 모델의 가중치를 업데이트할 최적화 함수 (Adam이 가장 무난합니다)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

사용 중인 디바이스: cpu


In [19]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()  # 모델을 학습 모드로 설정 (Dropout 등이 활성화됨)
    total_loss = 0.0
    
    for batch_idx, (batch_user, batch_item, batch_rating) in enumerate(dataloader):
        # 데이터를 GPU(또는 CPU)로 이동
        batch_user = batch_user.to(device)
        batch_item = batch_item.to(device)
        batch_rating = batch_rating.to(device)
        
        # 1. 기울기 초기화 (이전 배치의 기울기가 누적되지 않도록)
        optimizer.zero_grad()
        
        # 2. 순전파 (Forward Pass): 예측 평점 계산
        # 이 부분이 작성하신 DeepCoNN 클래스의 forward 함수를 실행하는 부분입니다.
        predictions = model(batch_user, batch_item)
        
        # 3. 오차 계산 (실제 평점과 예측 평점의 차이)
        loss = criterion(predictions, batch_rating)
        
        # 4. 역전파 (Backward Pass): 기울기 계산
        loss.backward()
        
        # 5. 가중치 업데이트
        optimizer.step()
        
        total_loss += loss.item()
        
        # 진행 상황 출력
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")
            
    avg_loss = total_loss / len(dataloader)
    print(f"==== Epoch {epoch+1} 완료 | 평균 Loss: {avg_loss:.4f} ====\n")

Epoch [1/10], Step [100/198], Loss: 4.3914


KeyboardInterrupt: 

In [ ]:
"""from gensim import models

model = KeyedVectors.load_word2vec_format("C:/Users/hanan/Downloads/cc.ko.300.vec.gz",binary=False,limit=20000)
for w, sim in model.similar_by_word('존맛', 10):
    print(f'{w}: {sim}')
"""

'from gensim import models\n\nmodel = KeyedVectors.load_word2vec_format("C:/Users/hanan/Downloads/cc.ko.300.vec.gz",binary=False,limit=20000)\nfor w, sim in model.similar_by_word(\'존맛\', 10):\n    print(f\'{w}: {sim}\')\n'